In [7]:
import pandas as pd
import os
# import wandb
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, matthews_corrcoef
from transformers import (
    AutoModel,
    AutoTokenizer,
    # DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from datasets import Dataset
from accelerate import Accelerator
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
import pickle
from saveAndLoad import *
import gc

In [8]:
checkpoint = 'facebook/esm2_t30_150M_UR50D'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint).to('cuda')
model.eval()

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t30_150M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EsmModel(
  (embeddings): EsmEmbeddings(
    (word_embeddings): Embedding(33, 640, padding_idx=1)
    (dropout): Dropout(p=0.0, inplace=False)
    (position_embeddings): Embedding(1026, 640, padding_idx=1)
  )
  (encoder): EsmEncoder(
    (layer): ModuleList(
      (0-29): 30 x EsmLayer(
        (attention): EsmAttention(
          (self): EsmSelfAttention(
            (query): Linear(in_features=640, out_features=640, bias=True)
            (key): Linear(in_features=640, out_features=640, bias=True)
            (value): Linear(in_features=640, out_features=640, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (rotary_embeddings): RotaryEmbedding()
          )
          (output): EsmSelfOutput(
            (dense): Linear(in_features=640, out_features=640, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (LayerNorm): LayerNorm((640,), eps=1e-05, elementwise_affine=True)
        )
        (intermediate): EsmIntermediate(
  

In [9]:
seqs = pickleLoad('../../aa/canonical_mut.pkl')
refs = pickleLoad('../../aa/canonical_ref.pkl')

loading data from ../../aa/canonical_mut.pkl
loading data from ../../aa/canonical_ref.pkl


In [10]:
mutSeqSet = set(seqs)
len(mutSeqSet)

656995

In [11]:
refSeqSet = set(refs)
len(refSeqSet)

1433

In [12]:
with torch.no_grad():
    # idx = 569032
    idx = 0
    print(len(seqs[idx]))
    inputs = tokenizer(seqs[idx])
    ids = torch.tensor([inputs['input_ids']]).to('cuda')
    print(ids)
    att = torch.tensor([inputs['attention_mask']]).to('cuda')
    output = model(ids, attention_mask=att)
    print(len(ids))
output['last_hidden_state'][0][0].shape

189
tensor([[ 0, 20, 11,  9, 19, 15,  4,  7,  7,  7,  6,  5, 23,  6,  7,  6, 15,  8,
          5,  4, 11, 12, 16,  4, 12, 16, 17, 21, 18,  7, 13,  9, 19, 13, 14, 11,
         12,  9, 13,  8, 19, 10, 15, 16,  7,  7, 12, 13,  6,  9, 11, 23,  4,  4,
         13, 12,  4, 13, 11,  5,  6, 16,  9,  9, 19,  8,  5, 20, 10, 13, 16, 19,
         20, 10, 11,  6,  9,  6, 18,  4, 23,  7, 18,  5, 12, 17, 17, 11, 15,  8,
         18,  9, 13, 12, 21, 21, 19, 10,  9, 16, 12, 15, 10,  7, 15, 13,  8,  9,
         13,  7, 14, 20,  7,  4,  7,  6, 17, 15, 23, 13,  4, 14,  8, 10, 11,  7,
         13, 11, 15, 16,  5, 16, 13,  4,  5, 10,  8, 19,  6, 12, 14, 18, 12,  9,
         11,  8,  5, 15, 11, 10, 16, 10,  7,  9, 13,  5, 18, 19, 11,  4,  7, 10,
          9, 12, 10, 16, 19, 10,  4, 15, 15, 12,  8, 15,  9,  9, 15, 11, 14,  6,
         23,  7, 15, 12, 15, 15, 23, 12, 12, 20,  2]], device='cuda:0')
1


torch.Size([640])

In [13]:
mean_pool = lambda x: np.mean(x, 0)
ref_idx_map = pickleLoad('../../aa/idxMap_canonical_mut_to_ref.pkl')
idx = 569032
with torch.no_grad():
    print(len(seqs[idx]))
    inputs = tokenizer(seqs[idx])
    ids = torch.tensor([inputs['input_ids']]).to('cuda')
    att = torch.tensor([inputs['attention_mask']]).to('cuda')
    output0 = model(ids, attention_mask=att)
    output0 = output0['last_hidden_state'].detach().to('cpu').numpy()[0][0]
    # output0 = mean_pool(output0['last_hidden_state'].detach().to('cpu').numpy()[0])

with torch.no_grad():
    idx = ref_idx_map[idx]
    print(len(refs[idx]))
    inputs = tokenizer(refs[idx])
    ids = torch.tensor([inputs['input_ids']]).to('cuda')
    att = torch.tensor([inputs['attention_mask']]).to('cuda')
    output1 = model(ids, attention_mask=att)
    output1 = output1['last_hidden_state'].detach().to('cpu').numpy()[0][0]
    # output1 = mean_pool(output1['last_hidden_state'].detach().to('cpu').numpy()[0])
    print(output1.shape)

print(F.cosine_similarity(torch.tensor(output0).unsqueeze(0),torch.tensor(output1).unsqueeze(0)))

for i,j in zip(output0[:50],output1[:50]):
    print(i,j)

loading data from ../../aa/idxMap_canonical_mut_to_ref.pkl
14509


OutOfMemoryError: CUDA out of memory. Tried to allocate 15.69 GiB. GPU 0 has a total capacty of 79.22 GiB of which 15.64 GiB is free. Process 2617587 has 62.13 GiB memory in use. Including non-PyTorch memory, this process has 1.44 GiB memory in use. Of the allocated memory 822.42 MiB is allocated by PyTorch, and 51.58 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [36]:
output['last_hidden_state'].shape

torch.Size([1, 14511, 640])

In [37]:
output['pooler_output'].shape

torch.Size([1, 640])

In [29]:
type(output['pooler_output'].detach().to('cpu').numpy()[0][0])

numpy.float32

In [15]:
max=5
n=0
longest = 0
longest_i = 0
lens = set()
for ni,i in enumerate(seqs):
    if i is None: continue
    lens.add(len(i))
    if len(i)>longest:
        longest = len(i)
        longest_i = ni
    if 900<len(i)<1000:
        if n<max:
            print(ni, len(i))
            n+=1
print('\n',longest,longest_i,'\n')
print(sorted(list(lens))[-15:])


9 976
54 928
57 976
59 976
83 976

 14511 568862 

[5809, 5883, 5889, 5890, 8789, 8794, 8795, 8796, 8797, 8798, 14506, 14507, 14508, 14509, 14511]


In [12]:
emb_mut = np.load('../../aa/canonical_mut_embeddings_esm2.npy')
emb_ref = np.load('../../aa/canonical_ref_embeddings_esm2.npy')

In [13]:
emb_mut[0][:5]

array([-0.07785489,  0.14835379,  0.14564785, -0.13385764, -0.33384255],
      dtype=float32)

In [14]:
emb_ref[0][:5]

array([0.13804878, 0.09532265, 0.0614852 , 0.18461367, 0.14487804],
      dtype=float32)

In [24]:
import pandas as pd
import os
# import wandb
import numpy as np
import torch
import torch.nn as nn
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, matthews_corrcoef
from transformers import (
    AutoModel,
    AutoTokenizer,
    # DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from datasets import Dataset
from accelerate import Accelerator
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
import pickle
from saveAndLoad import *
from tqdm import tqdm

muts = pickleLoad('../../aa/canonical_mut.pkl')
refs = pickleLoad('../../aa/canonical_ref.pkl')
data = pickleLoad('../../data_processing/consolidated_data.pkl')
tumors = pickleLoad('../../aa/tumors.pkl') 
        #0:concat_ref
        #1:concat_mut
        #2:iso_ref
        #3:iso_mut
        #4:canonical_ref [115] is None
        #5:canonical_mut [39] is None
        #6:data
mut2ref = pickleLoad('../../aa/idxMap_canonical_mut_to_ref.pkl')

def diff_indices(shorter, longer):
    # Ensure we treat the longer string as 'mutated' for uniform handling of insertions/deletions
    if shorter==longer: return None

    # Find the starting index where the two strings differ
    start = next((i for i, (o, m) in enumerate(zip(shorter, longer)) if o != m), len(shorter))

    # Calculate the length difference 
    len_diff = len(longer) - len(shorter)

    if len_diff == 0:
        # Find the end index of the replacement by comparing substrings starting from 'start'
        end = start + next((i for i, (o, m) in enumerate(zip(shorter[start:], longer[start:])) if o == m), len(shorter) - start)
        return start, end

    # Return the range of indices as 'insertion' or 'deletion'
    return start, start + len_diff

device = 'cuda:1'
checkpoint = 'facebook/esm2_t30_150M_UR50D'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint)
model.to(device)
model.eval()

mut_seqs = []
ref_seqs = []

for seq in muts: 
    if seq is None: 
        mut_seqs.append('')
        continue
    seq = seq.split('*')[0]
    mut_seqs.append(seq)

for seq in refs: 
    if seq is None: 
        ref_seqs.append('')
        continue
    seq = seq.split('*')[0]
    ref_seqs.append(seq)

mut_embeddings = np.load('/data/dandreas/SomaticMutationsLLM/aa/canonical_mut_average_embeddings_esm2.npy')
print('getting embeddings')
load_val = lambda x: torch.tensor([x]).to(device)

all_seqs = []
count = 0
for i,mut in tqdm(enumerate(mut_seqs), total = len(mut_seqs)):
    ref = ref_seqs[mut2ref[i]]
    if mut == '' or ref == '': 
        all_seqs.append(('',0,0,False))
        continue
    if mut == ref: 
        all_seqs.append(('',0,0,False))
        count+=1
        continue
    deletion = len(ref) > len(mut)
    seq = ref if deletion else mut
    (shorter,longer) = (mut,ref) if deletion else (ref,mut)
    start,end = diff_indices(shorter,longer)
    all_seqs.append((seq,start,end,deletion))
print(count, 'silent mutations')

for i,(seq,start,end,deletion) in tqdm(enumerate(all_seqs), total = len(all_seqs)):
    if all(mut_embeddings[i]==0): 
        if seq=='': continue
        mut_len = end-start
        with torch.no_grad():
            inputs = tokenizer(seq)
            ids = load_val(inputs['input_ids'])
            att = load_val(inputs['attention_mask'])
            output = model(ids, attention_mask=att)
            embedding = output['last_hidden_state'][0][start:end].detach().to('cpu').numpy()
            if mut_len>1: embedding = np.mean(embedding, axis = 0, keepdims = True)
            if deletion: embedding = -embedding
            mut_embeddings[i] = embedding
# np.save('/data/dandreas/SomaticMutationsLLM/aa/canonical_mut_average_embeddings_esm2.npy',mut_embeddings)



loading data from ../../aa/canonical_mut.pkl
loading data from ../../aa/canonical_ref.pkl
loading data from ../../data_processing/consolidated_data.pkl
loading data from ../../aa/tumors.pkl
loading data from ../../aa/idxMap_canonical_mut_to_ref.pkl


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t30_150M_UR50D and are newly initialized: ['esm.pooler.dense.weight', 'esm.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


getting embeddings


  1%|          | 3312/656995 [00:00<00:24, 26911.51it/s]


TypeError: cannot unpack non-iterable NoneType object

In [56]:
##DIFF INDICES TESTS

# Test cases
print(diff_indices('ACGT', 'ACGTTT'))  # Expected output: ('insertion', [4, 5])
print(diff_indices('ACGT', 'ACGG'))  # Expected output: ('deletion', [4, 5])
print(diff_indices('ACGT', 'ACGT'))  # Expected output: ('replacement', [4])
print(diff_indices('ACCGT','ACCCGT'))
print(diff_indices('TACCCGT','TTACCCGT'))
print(diff_indices('TACCCGT','GTACCCGT'))


[4, 5]
[3]
None
[3]
[1]
[0]


In [19]:
for i in data:
    assert i[-2] != i[-4]